In [1]:
import os
import numpy as np
import pandas as pd
import datetime

import numpy as np
import pandas as pd
import string
import re
import matplotlib.pyplot as plt
import math
from matplotlib import rc
from sklearn.model_selection import train_test_split
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
import matplotlib.ticker as ticker
from math import sqrt

import plotly.express as px 
import matplotlib.pyplot as plt
import warnings

from scipy.spatial.distance import cdist 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler, MinMaxScaler

---
### Import Data

In [3]:
df_last_artists = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/hetrec2011-lastfm-2k/artists.dat', sep='\t', usecols=['id', 'name'])
# df_last_tags = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/hetrec2011-lastfm-2k/tags.dat', sep='\t')
df_last_plays = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/hetrec2011-lastfm-2k/user_artists.dat', sep='\t')
# df_last_user_friends = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/hetrec2011-lastfm-2k/user_friends.dat', sep='\t')
# df_last_user_taggedartists = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/hetrec2011-lastfm-2k/data_w_genres.csv')
# df_last_user_taggedartists_timestamps = pd.read_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/hetrec2011-lastfm-2k/data_w_genres.csv')


df_last_artists

,id,name
0,1,MALICE MIZER
1,2,Diary of Dreams
2,3,Carpathian Forest
3,4,Moi dix Mois
4,5,Bella Morte
...,...,...
17627,18741,Diamanda Galás
17628,18742,Aya RL
17629,18743,Coptic Rain
17630,18744,Oz Alchemist


In [ ]:


test_dataset = pd.DataFrame(columns=cols)

cols = ["song", "songID", "artist", "album", "genre", "play_count", "click_count"]



test_data1 = [["Soul Food", 1, "Mat Zo", "Self Assemble", "Electronic", 82294, 555],
              ["The Enemy", 2, "Mat Zo", "Self Assemble", "Electronic", 42875, 777],
              ["Everytime We Touch", 3, "Cascada", "Everytime We Touch", "Electronic", 1756387293, 9872342342],
              ["Something in the Way", 4, "Nirvana", "Nevermind", "Grunge", 171857763, 223746531],
              ["heat", 5, "53 Thieves", "after hours", "Electronic", 88342, 101527],
              ["Another Life", 6, "Jadu Heart", "Hyper Romance", "Electronic", 17009, 28032],
              ["Numb", 7, "Elderbrook", "Why Do We Shake In The Cold?", "Electronic", 441070, 565088],
              ["Control", 8, "Emmit Fenn", "The Last Dance", "Electronic", 267650, 310823],
              ["Fire for You", 9, "Cannons", "Shadows", "Electronic", 32199, 37825],
              ["Dope", 10, "Demeter", "Dope", "Electronic", 1780336, 2138853]]

for i in range(len(test_data1)):
    
    to_append = dict(zip(cols, test_data1[i]))
    test_dataset = test_dataset.append(to_append, ignore_index=True)

test_dataset.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/test_dataset_small_1.csv', index=False)


In [4]:
df_last_plays

,userID,artistID,weight
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983
...,...,...,...
92829,2100,18726,337
92830,2100,18727,297
92831,2100,18728,281
92832,2100,18729,280


Let's merge the dataframes together to make things cleaner:

In [5]:
ap = pd.merge(
    df_last_artists, df_last_plays,
    how='inner',
    left_on='id',
    right_on='artistID'
)

ap = ap.rename(columns={'weight': 'playCount'})

In [6]:
ap

,id,name,userID,artistID,playCount
0,1,MALICE MIZER,34,1,212
1,1,MALICE MIZER,274,1,483
2,1,MALICE MIZER,785,1,76
3,2,Diary of Dreams,135,2,1021
4,2,Diary of Dreams,257,2,152
...,...,...,...,...,...
92829,18741,Diamanda Galás,454,18741,301
92830,18742,Aya RL,454,18742,294
92831,18743,Coptic Rain,454,18743,287
92832,18744,Oz Alchemist,454,18744,286


In [7]:
artist_rank = ap.groupby(['name']) \
  .agg({'userID' : 'count', 'playCount' : 'sum'}) \
  .rename(columns={"userID" : 'totalUniqueUsers', "playCount" : "totalArtistPlays"}) \
  .sort_values(['totalArtistPlays'], ascending=False)

artist_rank['avgUserPlays'] = artist_rank['totalArtistPlays'] / artist_rank['totalUniqueUsers']

In [8]:
ap = ap.join(artist_rank, on="name", how="inner") \
  .sort_values(['playCount'], ascending=False)

In [9]:
ap

,id,name,userID,artistID,playCount,totalUniqueUsers,totalArtistPlays,avgUserPlays
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376
35843,792,Thalía,2071,792,324663,26,350035,13462.884615
27302,511,U2,1094,511,320725,185,493024,2664.994595
8152,203,Blur,1905,203,257978,114,318221,2791.412281
26670,498,Paramore,1664,498,227829,399,963449,2414.659148
...,...,...,...,...,...,...,...,...
38688,913,Destiny's Child,1810,913,1,83,34746,418.626506
32955,697,Sia,1290,697,1,56,27597,492.803571
71811,4988,Chris Spheeris,510,4988,1,5,3106,621.200000
91319,17080,Haylie Duff,1851,17080,1,1,1,1.000000


In [11]:
df_test = ap.head(10)

In [12]:
df_test

,id,name,userID,artistID,playCount,totalUniqueUsers,totalArtistPlays,avgUserPlays
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376
35843,792,Thalía,2071,792,324663,26,350035,13462.884615
27302,511,U2,1094,511,320725,185,493024,2664.994595
8152,203,Blur,1905,203,257978,114,318221,2791.412281
26670,498,Paramore,1664,498,227829,399,963449,2414.659148
21191,378,Evanescence,1146,378,203165,226,513476,2272.017699
14165,292,Christina Aguilera,514,292,176133,407,1058405,2600.503686
33309,701,Shakira,1983,701,172496,319,688529,2158.398119
33127,701,Shakira,757,701,169596,319,688529,2158.398119
31835,679,Glee Cast,1086,679,165902,249,506453,2033.947791


In [13]:
df_test.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/last_merged_test_v1.csv', index=False)

In [15]:
ap.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/last_merged_full_v1.csv', index=False)

---
### Data Engineering





In [16]:
df_merged_encoded = ap.copy()

In [17]:
df_merged_encoded.name = LabelEncoder().fit_transform(df_merged_encoded.name.values.reshape(-1,1))
df_merged_encoded

C:\Users\awrg\Programs\Anaconda\envs\thesis\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,id,name,userID,artistID,playCount,totalUniqueUsers,totalArtistPlays,avgUserPlays
2800,72,3998,1642,72,352698,282,1301308,4614.567376
35843,792,14012,2071,792,324663,26,350035,13462.884615
27302,511,15654,1094,511,320725,185,493024,2664.994595
8152,203,2092,1905,203,257978,114,318221,2791.412281
26670,498,11069,1664,498,227829,399,963449,2414.659148
...,...,...,...,...,...,...,...,...
38688,913,4039,1810,913,1,83,34746,418.626506
32955,697,12913,1290,697,1,56,27597,492.803571
71811,4988,3000,510,4988,1,5,3106,621.200000
91319,17080,6352,1851,17080,1,1,1,1.000000


In [18]:
df_merged_encoded_test = df_merged_encoded.head(10)

In [19]:
df_merged_encoded_test.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/last_merged_encoded_test_v1.csv', index=False)
df_merged_encoded.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/last_merged_encoded_v1.csv', index=False)

---

In [20]:
df_merged_scaled = ap.copy()

df_merged_scaled.id = MinMaxScaler().fit_transform(df_merged_scaled.id.values.reshape(-1,1))
df_merged_scaled.name = LabelEncoder().fit_transform(df_merged_scaled.name.values.reshape(-1,1))
df_merged_scaled.name = MinMaxScaler().fit_transform(df_merged_scaled.name.values.reshape(-1,1))
df_merged_scaled.userID = MinMaxScaler().fit_transform(df_merged_scaled.userID.values.reshape(-1,1))
df_merged_scaled.artistID = MinMaxScaler().fit_transform(df_merged_scaled.artistID.values.reshape(-1,1))
df_merged_scaled.playCount = StandardScaler().fit_transform(df_merged_scaled.playCount.values.reshape(-1,1))
df_merged_scaled.totalUniqueUsers = StandardScaler().fit_transform(df_merged_scaled.totalUniqueUsers.values.reshape(-1,1))
df_merged_scaled.totalArtistPlays = StandardScaler().fit_transform(df_merged_scaled.totalArtistPlays.values.reshape(-1,1))
df_merged_scaled.avgUserPlays = StandardScaler().fit_transform(df_merged_scaled.avgUserPlays.values.reshape(-1,1))







    
df_merged_scaled

C:\Users\awrg\Programs\Anaconda\envs\thesis\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


,id,name,userID,artistID,playCount,totalUniqueUsers,totalArtistPlays,avgUserPlays
2800,0.003788,0.226760,0.781697,0.003788,93.821497,1.590902,4.371100,5.116137
35843,0.042200,0.794737,0.986177,0.042200,86.348091,-0.487206,0.875301,16.815652
27302,0.027209,0.887868,0.520496,0.027209,85.298322,0.803494,1.400766,2.538353
8152,0.010777,0.118655,0.907054,0.010777,68.571596,0.227144,0.758388,2.705506
26670,0.026515,0.627815,0.792183,0.026515,60.534653,2.540662,3.129514,2.207352
...,...,...,...,...,...,...,...,...
38688,0.048656,0.229085,0.861773,0.048656,-0.198396,-0.024502,-0.283344,-0.431863
32955,0.037132,0.732403,0.613918,0.037132,-0.198396,-0.243678,-0.309615,-0.333784
71811,0.266058,0.170155,0.242135,0.266058,-0.198396,-0.657676,-0.399617,-0.164015
91319,0.911172,0.360275,0.881316,0.911172,-0.198396,-0.690146,-0.411027,-0.984062


In [21]:
df_merged_scaled_test = df_merged_scaled.head(10)

In [22]:
df_merged_scaled_test.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/last_merged_scaled_test_v1.csv', index=False)
df_merged_scaled.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/last_merged_scaled_v1.csv', index=False)

In [24]:
# df_spot.to_csv('C:/Users/awrg/Documents/Delft/Thesis/Projects/Datasets/Engineered/spotify_encoded_v1.csv')

---
# Preprocessing

In [106]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error


NameError: name 'df_merged_test' is not defined

In [66]:
pc = ap.playCount

pc_scaled = (pc - pc.min()) / (pc.max() - pc.min())

ap = ap.assign(playCountScaled=pc_scaled)

In [67]:
ap

,id,name,userID,artistID,playCount,totalUniqueUsers,totalArtistPlays,avgUserPlays,playCountScaled
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376,1.000000
35843,792,Thalía,2071,792,324663,26,350035,13462.884615,0.920513
27302,511,U2,1094,511,320725,185,493024,2664.994595,0.909347
8152,203,Blur,1905,203,257978,114,318221,2791.412281,0.731441
26670,498,Paramore,1664,498,227829,399,963449,2414.659148,0.645960
...,...,...,...,...,...,...,...,...,...
38688,913,Destiny's Child,1810,913,1,83,34746,418.626506,0.000000
32955,697,Sia,1290,697,1,56,27597,492.803571,0.000000
71811,4988,Chris Spheeris,510,4988,1,5,3106,621.200000,0.000000
91319,17080,Haylie Duff,1851,17080,1,1,1,1.000000,0.000000


In [69]:
ratings_df = ap.pivot(
    index='userID',
    columns='artistID',
    values='playCountScaled'
)

ratings = ratings_df.fillna(0).values
ratings

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.00115397, ..., 0.        , 0.        ,
        0.        ]])

In [70]:
sparsity = float(len(ratings.nonzero()[0]))
sparsity /= (ratings.shape[0] * ratings.shape[1])
sparsity *= 100
print('{:.2f}%'.format(sparsity))

0.28%


In [72]:
MIN_USER_RATINGS = 35
DELETE_RATING_COUNT = 15

def train_test_split(ratings):

    validation = np.zeros(ratings.shape)
    train = ratings.copy()

    for user in np.arange(ratings.shape[0]):
        if len(ratings[user,:].nonzero()[0]) >= MIN_USER_RATINGS:
            val_ratings = np.random.choice(
              ratings[user, :].nonzero()[0],
              size=DELETE_RATING_COUNT,
              replace=False
            )
            train[user, val_ratings] = 0
            validation[user, val_ratings] = ratings[user, val_ratings]
    return train, validation

In [73]:
train, val = train_test_split(ratings)

In [96]:
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten()
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return np.sqrt(mean_squared_error(prediction, ground_truth))

In [97]:
def fit(self, X_train, X_val):
    m, n = X_train.shape

    self.P = 3 * np.random.rand(self.n_latent_features, m)
    self.Q = 3 * np.random.rand(self.n_latent_features, n)

    self.train_error = []
    self.val_error = []

    users, items = X_train.nonzero()

    for epoch in range(self.n_epochs):
        for u, i in zip(users, items):
            error = X_train[u, i] - self.predictions(self.P[:,u], self.Q[:,i])
            self.P[:, u] += self.learning_rate * \
               (error * self.Q[:, i] - self.lmbda * self.P[:, u])
            self.Q[:, i] += self.learning_rate * \
               (error * self.P[:, u] - self.lmbda * self.Q[:, i])

        train_rmse = rmse(self.predictions(self.P, self.Q), X_train)
        val_rmse = rmse(self.predictions(self.P, self.Q), X_val)
        self.train_error.append(train_rmse)
        self.val_error.append(val_rmse)

In [98]:
def predictions(self, P, Q):
    return np.dot(P.T, Q)

In [99]:
class Recommender:
  
    def __init__(self, n_epochs=200, n_latent_features=3, lmbda=0.1, learning_rate=0.001):
        self.n_epochs = n_epochs
        self.n_latent_features = n_latent_features
        self.lmbda = lmbda
        self.learning_rate = learning_rate
  
    def predictions(self, P, Q):
        return np.dot(P.T, Q)
  
    def fit(self, X_train, X_val):
        m, n = X_train.shape

        self.P = 3 * np.random.rand(self.n_latent_features, m)
        self.Q = 3 * np.random.rand(self.n_latent_features, n)

        self.train_error = []
        self.val_error = []

        users, items = X_train.nonzero()

        for epoch in range(self.n_epochs):
            for u, i in zip(users, items):
                error = X_train[u, i] - self.predictions(self.P[:,u], self.Q[:,i])
                self.P[:, u] += self.learning_rate * (error * self.Q[:, i] - self.lmbda * self.P[:, u])
                self.Q[:, i] += self.learning_rate * (error * self.P[:, u] - self.lmbda * self.Q[:, i])

            train_rmse = rmse(self.predictions(self.P, self.Q), X_train)
            val_rmse = rmse(self.predictions(self.P, self.Q), X_val)
            self.train_error.append(train_rmse)
            self.val_error.append(val_rmse)

        return self
  
    def predict(self, X_train, user_index):
        y_hat = self.predictions(self.P, self.Q)
        predictions_index = np.where(X_train[user_index, :] == 0)[0]
        return y_hat[user_index, predictions_index].flatten()

In [100]:
def predict(self, X_train, user_index):
    y_hat = self.predictions(self.P, self.Q)
    predictions_index = np.where(X_train[user_index, :] == 0)[0]
    return y_hat[user_index, predictions_index].flatten()

In [107]:
recommender = Recommender().fit(train, val)


KeyboardInterrupt



In [93]:
user_id = 1236
user_index = ratings_df.index.get_loc(user_id)
predictions_index = np.where(train[user_index, :] == 0)[0]

rating_predictions = recommender.predict(train, user_index)

TypeError: predict() missing 1 required positional argument: 'user_index'

In [83]:
existing_ratings_index = np.where(train[user_index, :] > 0)[0]
existing_ratings = train[user_index, existing_ratings_index]

create_artist_ratings(
  artists,
  existing_ratings_index,
  existing_ratings
)

NameError: name 'create_artist_ratings' is not defined